### http://www.engear.net/wp/topic-modeling-gensimpython/ 
### https://github.com/smilemango/lda_study/blob/master/lda_with_gensim.ipynb

# Topic Modeling with Gensim

## 1. 소개
자연언어 처리의 주요 어플리케이션중 하나는 커다란 볼륨의 텍스트로부터 사람들이 어떤 주제에 대해서 이야기하는지 자동으로 토픽을 추출하는 것입니다. 큰 텍스트의 예로는 소셜 미디어의 피드, 호텔, 영화등의 고객 리부, 사용자 피드백, 뉴스기사, 고객불만사항 전자메일 등이 있습니다. 사람들이 무엇을 원하는가를 알고 그들의 문제와 의견을 이해하는것은 사업이나, 관리, 정치 캠페인에 유용합니다. 그리고 그렇게 많은 양을 직접 읽고 주제를 이해하는 것은 정말 어렵습니다. 따라서 텍스트를 읽고 논의된 주제를 출력할 수 있는 자동화된 알고리즘이 필요합니다. 이 튜토리얼에서는 ’20개의 뉴스그룹’ 데이터세트의 실제 예제를 사용하고 LDA를 사용하여 있는 그대로의 논의된 주제를 추출해낼 것입니다. Gensim 패키지에 있는 Latent Dirichlet Allocation(LDA)를 사용할 것이고, (Gensim을 통해) Mallet’s의 구현도 함께 사용할 것입니다. Mallet은 LDA를 효율적으로 구현하였습니다. 더 빠르게 수행되고 더 좋은 토픽을 제시해주는 것으로 알려져있습니다. 우리는 또한 각 토픽의 양과 기여도 비율을 추출하여 얼마나 중요한 주제인지를 알아보도록 하겠습니다.

시작해봅시다.

## 2. 사전 준비 – nltk stopwords와 spacy 모델 다운받기
텍스트 전처리를 위하여 NLTK의 stopwords와 spacy의 en 모델이 필요합니다. 나중에 우리는 표준형변환(lemmatization)을 위하여 spacy 모델을 사용할 것입니다. 표준형변환(Lemmatization)은 근원 단어로 변환하는 것입니다. 예를들어 ‘machines’의 lemma 단어는 ‘machine’입니다. 마찬가지로, ‘walking’ -> ‘walk’,’mice’ -> ‘mouse’ 등입니다.



In [8]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

#### Run in terminal or command prompt 
python3 -m spacy download en

# 3. Import Packages
이 튜토리얼에서 사용할 핵심 패키지는 re , gensim , spacy , pyLDAvis  입니다. 이외에 우리는 matplotlib , numpy , pandas 를 사용하여 데이터를 다루고 시각화 할것입니다. 이것들을 import 합시다.

In [11]:
import re 
import numpy as np
import pandas as pd
from pprint import pprint

In [14]:
#Gensism
import gensim
import gensim.corpora as corpora 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [18]:
#spacy for lemmatization 
#spaCy는 파이썬의 자연어처리를 위한 오픈 소스 기반 라이브러리다.
#아직 한국어는 지원하지 않지만, 텍스트 전처리에서 좋은 성능을 보여줌
import spacy

In [24]:
#Plotting tools
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt 
%matplotlib inline

In [25]:
#Enable logging for gensim -optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)', level=logging.ERROR)

In [27]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 4. LDA는 무엇을 합니까?
토픽 모델링에 대한 LDA의 접근방식은 각 도큐먼트를 특정 비율의 토픽의 집합으로 가정합니다. 그리고 다시, 각 토픽은 다시 각 키워드의 일정 비율로 구성됩니다.알고리즘에 토픽의 개수를 입력하면, 문서 내에서의 토픽 분포화 토픽 안에서의 키워드 분포를 재정렬하여 토픽-키워드 분포의 적절한 구성을 얻을 수 있습니다. 토픽을 말할때, 실제로 그리고 어떻게 표현되는가? 토픽은 일반적으로 표현되는 지배적인 키워드의 모음입니다. 키워드를 살펴보는것만으로 주제가 무엇인지를 파악할 수 있습니다.

좋은 토픽 구분을 얻기 위한 핵심 요소는 다음과 같습니다.

1. 처리된 텍스트 데이터의 품질
2. 텍스트가 말하고자하는 토픽의 다양성
3. 토픽 모델링 알고리즘의 선택
4. 알고리즘에 공급된 주제수
5. 알고리즘 튜닝 파라미터
 

## 5. Stopwords(불용단어) 준비하기
앞서 우리는 stopwords를 다운로드 하였습니다. 이것을 import 하여 사용하도록 합시다.

In [31]:
#NLTK Stop words
from nltk.corpus import stopwords 
stop_words=stopwords.words('english')
stopwords.expend(['from','subject','re','edu','use'])

AttributeError: 'WordListCorpusReader' object has no attribute 'expend'

## 6. 뉴스그룹 데이터 임포트하기
본 실습을 위하여 ’20개 뉴스그룹’ 데이터를 사용할 것입니다. 이 데이터에는 20개의 서로 다른 주제로 구성된 약 11,000개의 뉴스그룹 게시물이 포함되어 있습니다. 이것은 newsgroups.json 에서 다운로드 가능합니다.

이것을 https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json  'pandas.read_json' 을 통해서 읽으며, 결과 데이터 세트는 보여지는 것 처럼 3개의 컬럼을 가지고 있습니다.

In [32]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


## 7. 이메일 주소와 줄바꿈 문자 제거
보는 것처럼 이메일, 줄바꿈 문자 및 불필요한 공백이 있어서 불편합니다. 이를 정규식을 사용하여 제거하도록 합시다.

In [33]:
# Convert to list
data = df.content.values.tolist()
 
# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
 
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
 
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
 
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


전자메일과 불필요한 공백을 제거하고나서도 여전히 지저분해보입니다. LDA에서 처리하기에는 부적합합니다. 토큰화를 통해 각 문장을 단어의 list로 분해하면서 처리 과정에서 지저분한 모든 문자를 지워야합니다.

이를 위해서 Gensim의 simple_process  가 적절합니다.

## 8. 단어 토큰화와 텍스트 클린업
각 문장의 구두점과 불필요한 문자를 제거하여, 단어의 list로 토크나이징합시다. 이작업에는 Gensim의 simple_preprocess() 가 훌륭합니다. 추가적으로 나는 구두점을 제거하기 위해서 deacc=True 로 설정하였습니다.

In [37]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) #deacc=True removes punctions
data_words=list(sent_to_words(data))
print(data_words[:1])   

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## 9. Bigram과 Trigram 모델 만들기
Bigram은 문서에서 자주 발생되는 2연 단어입니다. Tigram은 자주 발생하는 3연 단어입니다. 이 예에서는 ‘front_bumper’,’oil_leak’,’maryland_college_park’등이 있습니다. Gensim의 Phrases 모델은 bigram, trigram, quadgram 등 그 이상을 구현할 수 있습니다. Phrases에 대한 두가지 중요한 인수는 min_count및 임계값입니다. 이 매개변수의 값이 높을수록 단어가 바이그램으로 결합되는 것이 어렵습니다.

In [39]:
#Build the bigram and trigram models
bigram=gensim.models.Phrases(data_words,min_count=5,threshold=100) #higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

#Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod=gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


## 바이그램 모형이란?
만약 단어의 활용이 바로 전 단어에만 의존한다면 단어 열의 확률은 다음과 같다. 이러한 모형을 바이그램 모형 또는 마코프 모형(Markov Model)이라고 한다.

![Bigram](2.png)

## N그램 모형이란?
만약 단어의 활용이 바로 전  n−1 개의 단어에만 의존한다면 단어 열의 확률은 다음과 같다. 이러한 모형을 N그램 모형이라고 한다.

![Ngram](3.png)

## 10. 불용단어(Stopwords)를 제거하고, Bigram을 만들고 표준형변환
Bigram 모델이 준비되었습니다. 불용단어를 제거하고 Bigram을 만들고 표준화할 함수를 정의한뒤 순차적으로 실행합니다.

In [87]:
#Define functions for stopwords, bigrams, trigrams and lemmatization 
# lemmatization : 했습니다 > 했다 통일 시켜 주는 것 

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [82]:
!python -m spacy download en_core_web_sm

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-py3-none-any.whl size=12011743 sha256=4a647cbd1a5617f6acb7a90f7c7f7d2b682b3bce5d2a78b366c4f8d0c0045b7a
  Stored in directory: C:\Users\LENOVO\AppData\Local\Temp\pip-ephem-wheel-cache-z296n0zs\wheels\51\19\da\a3885266a3c241aff0ad2eb674ae058fd34a4870fef1c0a5a0
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [88]:
#순서대로 이 함수들을 수행합시다.

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # 해결 안되서 아래 코드로 ,,, 
nlp=en_core_web_sm.load(disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])                      

[['where', 'thing', 'subject', 'car', 'nntp_posting', 'host', 'rac_wam', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


## 11. 토픽 모델에 필요한 사전과 코퍼스 생성하기
LDA 토픽 모델의 두가지 주요 입력값은 사전(id2word)와 코퍼스 입니다. 이것들을 만들어 봅시다.

In [89]:
#Create Dictionary
#corpora 란 curpus의 복수형 
#https://m.blog.naver.com/PostView.nhn?blogId=pdc222&logNo=221360553916&proxyReferer=https:%2F%2Fwww.google.com%2F



id2word=corpora.Dictionary(data_lemmatized)사전 구축 

#Create Corpus 
texts=data_lemmatized

#Term Document Frequency
corpus=[id2word.doc2bow(text) for text in texts]

#View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]]


In [90]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 5),
  (6, 1),
  (7, 1),
  (8, 2),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1)],
 [(4, 2),
  (7, 2),
  (14, 1),
  (19, 1),
  (38, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 3),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 2),
  (57, 1),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 3),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 2),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 2),
  (82, 1)],
 [(6, 3),
  (7, 1),
  (15, 2),
  (16, 1),


Gensim 은 문서안의 각 단어에 대해서 유니크한 ID를 만듭니다. 위에 표시된 생성된 코퍼스는 (word_id, word_frequency)의 매핑입니다. 예를들어, 위의 (0,1)은 첫번째 문서에서 단어 ID 0이 한번 발생한다는 것을 의미합니다. 마찬가지로, id 1단어는 2번 나옵니다. 이것이 LDA 모델의 입력으로 사용됩니다. ID가 어떤 단어를 나타내는지 알고 싶으면 사전에 id를 키로 넘기면 알 수 있습니다.

In [92]:
id2word[0]

'addition'

In [93]:
#또는 사람이 이해할 수 있는 형태의 코퍼스를 만들 수 도 있다.
[[(id2word[id],freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_posting', 1),
  ('park', 1),
  ('production', 1),
  ('rac_wam', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('sport', 1),
  ('subject', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

## 12. 토픽 모델 만들기
이제 LDA 모델을 훈련하기 위한 모든 것을 준비했습니다. 코퍼스와 사전 외에도 토픽 개수를 제공해야합니다. 그 외에도, 알파와 에타는 토픽의 희소성에 영향을 주는 하이퍼파라미터입니다. Gensim docs에 따르면 기본값은 모두 ‘1.0/num_topics’ 을 기본값으로 사용합니다.

chunksize는 각 훈련 chunk에서 사용할 문서의 수입니다. update_every는 모델 매개변수를 업데이트해야하는 빈도를 결정하고, passes는 총 훈련 과정 수를 결정합니다.

In [94]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
id2word=id2word,
num_topics=20, #k=20
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto',
per_word_topics=True)

## 13. LDA 모델의 토픽 보기
위의 LDA 모델은 각 토픽이 키워드의 조합이고 각 키워드가 토픽에 일정한 가중치를 부여하는 20개의 주제로 구성됩니다.  lda_model.print_topics()  를 사용하여 각 토픽의 키워드와 각 키워드의 중요도(가중치)를 볼수 있습니다.

In [95]:
#Print the Keyword in the 10 topics 
pprint(lda_model.print_topics())
doc_lda=lda_model[corpus]

#해석 : 토픽 0은 car가 가중치 0.016으로 제일 높지만 그리 높은 수치는 아니다 

[(0,
  '0.054*"year" + 0.034*"first" + 0.026*"last" + 0.024*"day" + 0.023*"go" + '
  '0.023*"game" + 0.017*"start" + 0.017*"next" + 0.017*"win" + 0.015*"will"'),
 (1,
  '0.060*"use" + 0.036*"system" + 0.021*"program" + 0.020*"file" + '
  '0.017*"include" + 0.017*"information" + 0.014*"also" + 0.013*"window" + '
  '0.013*"available" + 0.012*"software"'),
 (2,
  '0.225*"drive" + 0.099*"driver" + 0.063*"speed" + 0.058*"bike" + '
  '0.046*"monitor" + 0.037*"mode" + 0.035*"normal" + 0.035*"ride" + '
  '0.030*"fast" + 0.030*"switch"'),
 (3,
  '0.061*"player" + 0.057*"sale" + 0.035*"learn" + 0.035*"goal" + '
  '0.031*"serious" + 0.030*"shall" + 0.029*"wing" + 0.025*"internal" + '
  '0.023*"purchase" + 0.022*"owner"'),
 (4,
  '0.147*"faith" + 0.072*"church" + 0.067*"christian" + 0.057*"deny" + '
  '0.035*"insist" + 0.031*"scripture" + 0.028*"benefit" + 0.025*"blind" + '
  '0.025*"statistic" + 0.019*"excuse"'),
 (5,
  '0.030*"car" + 0.026*"high" + 0.021*"space" + 0.019*"power" + 0.019*"price" '

## 14. 모델 난이도와 일관성 점수 계산하기
모델 난이도와 토픽 일관성은 토픽 모델이 얼마나 훌륭한지 판단할 수 있는 편리한 수단을 제공합니다. 내 경험에서는 특히 토픽 일관성 점수가 더 도움이 되었습니다.

In [98]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

#일관성 점수가 0.44확인 


Perplexity:  -12.844415762915416

Coherence Score:  0.4430427536589347


## 15. 토픽-키워드 시각화하기
LDA모델이 완성되었으므로 다음 다계는 생성된 토픽과 키워드를 검사하는 것입니다. pyLDAvis패키지의 대화형차트가 가장 훌륭하며 jupyter notebook과 잘 동작합니다.

In [99]:
# Visualize the topics
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#vis

C:\Users\LENOVO\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.338375  0.002780       1        1  34.027317
1     -0.260921  0.172178       2        1  11.381742
16    -0.278721 -0.061914       3        1  10.851387
13    -0.235055  0.314359       4        1   9.679710
7     -0.220290 -0.218088       5        1   9.378219
0     -0.211092 -0.189333       6        1   7.638811
5     -0.136783 -0.098811       7        1   6.327026
3      0.077836  0.031740       8        1   1.520038
17     0.121715  0.005432       9        1   1.437257
8      0.029287  0.080481      10        1   1.293244
2      0.107130  0.019552      11        1   1.008071
6      0.121129  0.002097      12        1   0.884995
14     0.131959  0.006544      13        1   0.847984
11     0.118736 -0.018539      14        1   0.815660
10     0.109922 -0.047994      15        1   0.760766
4      0.140074  0.004161      16        1   0.629895
19     0.166574  0.000688      17        1   0.527668
15     0.179707 -0.006757      18        1   0.351064
12     0.188456  0.000730      19        1   0.349938
9      0.188711  0.000693      20        1   0.289197, topic_info=     Category          Freq       Term         Total  loglift  logprob
19    Default  15080.000000       line  15080.000000   30.000  30.0000
199   Default  13034.000000      write  13034.000000   29.000  29.0000
176   Default  16113.000000      would  16113.000000   28.000  28.0000
171   Default  11937.000000        use  11937.000000   27.000  27.0000
36    Default   8114.000000    subject   8114.000000   26.000  26.0000
...       ...           ...        ...           ...      ...      ...
2125  Topic20      0.040101       jpeg      1.215876    2.434 -11.3191
337   Topic20      0.040102      tumor      1.215909    2.434 -11.3191
3781  Topic20      0.040102  diagnosis      1.215915    2.434 -11.3191
4480  Topic20      0.040102       tube      1.215913    2.434 -11.3191
3181  Topic20      0.040103       slow      1.215987    2.434 -11.3191

[858 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
474      12  0.992368  absolute
1215      3  0.948674    accept
1215      5  0.050596    accept
83        2  0.998868    access
1292      8  0.996555   acquire
...     ...       ...       ...
2657      5  0.423634   writing
2657     16  0.567124   writing
42        6  0.987154      year
42        7  0.012509      year
572       1  0.998248     young

[1060 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 2, 17, 14, 8, 1, 6, 4, 18, 9, 3, 7, 15, 12, 11, 5, 20, 16, 13, 10])

어떻게 pyLDAvis의 결과물을 추론할 수 있을까요? 왼쪽 표의 각 버블은 토픽을 나타냅니다. 버블이 클 수록 그 토픽이 더 일반적입니다.(역주:특성화되지 않고 여러 단어가 등장한다는 의미) 좋은 토픽 모델은 하나의 사분면에 클러스터되지 않고 차트 전체에 흩어져있는 상당히 크고 겹치지 않는 버블을 가질 것입니다. 너무 많은 토픽을 가진 모델으 일반적으로 차트의 한 영역에 클러스터된 작은 크기의 버블이 많이 겹치게 될 것입니다. 이제, 버블중 하나로 커서를 이동하면 오른쪽의 단어와 막대가 업데이트 됩니다. 이 단언느 선택한 주제를 형성하게 하는 주도적인 키워드입니다. 우리는 성공적으로 좋은 토픽모델을 만들었습니다.

문서 그대로의 토픽 수에 대한 사전 지식이 있기때문에 최상의 모델을 찾는것은 직관적이었습니다. 다음으로, Mallet의 LDA알고리즘을 사용하여 이 모델을 개선한다음, 큰 텍스트 코프스가 주어질 때 취적의 토픽 수에 도달하는 방법을 알아보겠습니다.